In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import KFold
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
data = pd.read_csv('dulieuthunho.csv')
data['Ngày'] = pd.to_datetime(data['Ngày'])

In [3]:
features = ['Mở cửa', 'Đóng cửa', 'Cao nhất', 'Thấp nhất', 'Trung bình', 'GD khớp lệnh KL']
data = data[['Ngày', 'Mã CK'] + features]
data = data.sort_values(by=['Mã CK', 'Ngày'])

In [4]:
def prepare_data(data, features):
    data_scaled = data.copy()
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled[features] = scaler.fit_transform(data[features])
    return data_scaled, scaler

In [5]:
def create_sequences(data, seq_length, feature_col):
    X, Y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        Y.append(data[i + seq_length][feature_col])
    return np.array(X), np.array(Y)

In [6]:
def train_lstm(X_train, Y_train, seq_length, epochs=100):
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X_train.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    epoch_range = tqdm(range(epochs), desc="Epochs")
    for epoch in epoch_range:
        model.fit(X_train, Y_train, epochs=4, batch_size=32, verbose=0)
        epoch_range.set_postfix({'loss': model.history.history['loss'][0]})

    return model

In [7]:
def evaluate_performance(model, test_data, scaler, seq_length):
    X_test, Y_test = create_sequences(test_data, seq_length, features.index('Đóng cửa'))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(features)))

    test_predict = model.predict(X_test)
    
    test_predict_transformed = np.zeros((test_predict.shape[0], len(features)))
    test_predict_transformed[:, features.index('Đóng cửa')] = test_predict.flatten()
    test_predict_transformed = scaler.inverse_transform(test_predict_transformed)[:, features.index('Đóng cửa')]

    Y_test_transformed = np.zeros((Y_test.shape[0], len(features)))
    Y_test_transformed[:, features.index('Đóng cửa')] = Y_test.flatten()
    Y_test_transformed = scaler.inverse_transform(Y_test_transformed)[:, features.index('Đóng cửa')]

    rmse = np.sqrt(mean_squared_error(Y_test_transformed, test_predict_transformed))
    mae = mean_absolute_error(Y_test_transformed, test_predict_transformed)
    mape = np.mean(np.abs((Y_test_transformed - test_predict_transformed) / Y_test_transformed)) * 100
    r2 = r2_score(Y_test_transformed, test_predict_transformed)

    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"R-squared: {r2}")


In [8]:
def predict_next_close(stock_data, seq_length, model, features, scaler):
    last_sequence = stock_data[features].values[-seq_length:]
    last_sequence = scaler.transform(last_sequence)
    last_sequence = np.expand_dims(last_sequence[:-1], axis=0)
    predicted_price = model.predict(last_sequence)
    predicted_price = np.concatenate([predicted_price, np.zeros((predicted_price.shape[0], len(features)-1))], axis=1)
    predicted_price = scaler.inverse_transform(predicted_price)
    return predicted_price[0][features.index('Đóng cửa')]

In [ ]:
models = {}
scalers = {}
seq_length = 20
epochs = 100

for symbol in data['Mã CK'].unique():
    stock_data = data[data['Mã CK'] == symbol].copy()
    stock_data_scaled, scaler = prepare_data(stock_data, features)

    if len(stock_data_scaled) < seq_length:
        print(f"Skipping {symbol} due to insufficient data")
        continue

    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(stock_data_scaled):
        train_data, test_data = stock_data_scaled.iloc[train_index], stock_data_scaled.iloc[test_index]

        if len(train_data) < seq_length or len(test_data) < seq_length:
            print(f"Skipping split for {symbol} due to insufficient sequence data")
            continue

        X_train, Y_train = create_sequences(train_data[features].values, seq_length, features.index('Đóng cửa'))
        X_test, Y_test = create_sequences(test_data[features].values, seq_length, features.index('Đóng cửa'))

        lstm_model = train_lstm(X_train, Y_train, seq_length, epochs)
        evaluate_performance(lstm_model, test_data[features].values, scaler, seq_length)

        models[symbol] = lstm_model
        scalers[symbol] = scaler

# Lưu mô hình và scaler
with open('models.pkl', 'wb') as file:
    pickle.dump(models, file)

with open('scalers.pkl', 'wb') as file:
    pickle.dump(scalers, file)

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:25<00:00,  3.90it/s, loss=0.00177]


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 393ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


RMSE: 1.7060855189382484
MAE: 1.6854331174045805
MAPE: 15.160732062396665
R-squared: -30.416382060774843


Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:32<00:00,  3.12it/s, loss=0.00375]


...

Epochs: 100%|██████████████████████████████████████████████████████████| 100/100 [00:26<00:00,  3.73it/s, loss=0.00133]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


RMSE: 8.80781561152233e-06
MAE: 8.80781561152233e-06
MAPE: 0.0007934969019389485
R-squared: nan


Epochs:  13%|███████▋                                                   | 13/100 [00:06<00:25,  3.37it/s, loss=0.00674]